Import Packages

In [6]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import operator

Files for Gephi

In [2]:
df = pd.read_csv(r'./dataset.csv')

df2 = df[["track_id","writers","track_name","track_pos"]]
df2_dict = df2.to_dict(orient='records')

d = []

for f in df2_dict:
    d.extend(f["writers"].split("-"))

set_d = set([a.strip() for a in d])
dict_vett_writers = []
new_dict_vect_writers = []
support = []
support2 = ""

for writer in set_d:
    for d in df2_dict:
        if writer in [a.strip() for a in d["writers"].split("-")]:
            support.append({"name":d["track_name"],"pos":d["track_pos"]})
            support2 += d["track_name"]+" - "+str(d["track_pos"])+","
    dict_vett_writers.append({"writer":writer,"tracks":support})
    new_dict_vect_writers.append({"writer":writer,"tracks":support2[:-1].split(",")})
    support = []
list_edges = []
l_e_new = []
list_weights = []

for dic in dict_vett_writers:
    for dic2 in dict_vett_writers:
        for song in dic["tracks"]:
            if song in dic2["tracks"] and dic["writer"] != dic2["writer"]:
                if {"source":dic["writer"],"target":dic2["writer"]} in list_edges:
                    index = list_edges.index({"source":dic["writer"],"target":dic2["writer"]})
                    list_weights[index] += (10-song["pos"])
                else:
                    common_tracks =[]
                    list_edges.append({"source":dic["writer"],"target":dic2["writer"]})
                    for t1 in dic["tracks"]:
                        if t1 in dic2["tracks"]:
                            common_tracks.append(t1)
                    l_e_new.append({"source":dic["writer"],"target":dic2["writer"],"tracks":common_tracks})
                    list_weights.append(1 + (10-song["pos"]))

for weight in list_edges:
    a = weight["source"]
    b = weight["target"]
    index = list_edges.index(weight)
    try:
        list_edges.remove({"source":b,"target":a})
        list_weights.remove(list_weights[index])
    except:
        pass

lista =list(set([str(a["writer"]).strip() for a in dict_vett_writers]))
print(True if ("M. Vicino") in lista else False)
'''for w in lista:
    if w not in [b["source"] for b in list_edges]:
        lista.remove(w)'''

True


'for w in lista:\n    if w not in [b["source"] for b in list_edges]:\n        lista.remove(w)'

In [23]:
#top songwriters per numero di canzoni
v_singers = ['D. Pace','F. Migliacci','M. Panzeri','Mogol','A. Cogliati']
tabella = []
for singer in v_singers:
    val_singer = 0
    val_placed = 0
    for i , row in df2.iterrows():
        if singer in [a.strip() for a in row["writers"].split("-")] and row["track_pos"] <= 1:
            val_singer += 1
        if singer in [a.strip() for a in row["writers"].split("-")]:
            val_placed += 1
    tabella.append({"writer":singer,"val_top3":val_singer, "val_placed":val_placed})
    print("fine "+singer)

print(tabella)

fine D. Pace
fine F. Migliacci
fine M. Panzeri
fine Mogol
fine A. Cogliati
[{'writer': 'D. Pace', 'val_top3': 0, 'val_placed': 22}, {'writer': 'F. Migliacci', 'val_top3': 3, 'val_placed': 15}, {'writer': 'M. Panzeri', 'val_top3': 2, 'val_placed': 26}, {'writer': 'Mogol', 'val_top3': 4, 'val_placed': 16}, {'writer': 'A. Cogliati', 'val_top3': 2, 'val_placed': 11}]


In [39]:
v_singers = list(set([a["writer"] for a in dict_vett_writers]))
tabella = []
tabella2 = []
for singer in v_singers:
    val_singer = 0
    val_placed = 0
    for i , row in df2.iterrows():
        if singer in [a.strip() for a in row["writers"].split("-")] and row["track_pos"] <= 1:
            val_singer += 1
        if singer in [a.strip() for a in row["writers"].split("-")]:
            val_placed += 1
    if val_placed > 0 and val_singer > 1:
        tabella.append({"writer":singer,"val_agg":val_singer/val_placed , "val_top3":val_singer, "val_number":val_placed})
    #print("fine "+singer)

for singer in v_singers:
    val_singer = 0
    val_placed = 0
    for i , row in df2.iterrows():
        if singer in [a.strip() for a in row["writers"].split("-")] and row["track_pos"] <= 1:
            val_singer += 1
        if singer in [a.strip() for a in row["writers"].split("-")]:
            val_placed += 1
    tabella2.append({"writer":singer,"val_agg":val_singer/val_placed , "val_top3":val_singer, "val_number":val_placed})


print(tabella)
print(len(tabella))

[{'writer': 'A. Civai', 'val_agg': 0.3333333333333333, 'val_top3': 2, 'val_number': 6}, {'writer': 'M. Luca', 'val_agg': 1.0, 'val_top3': 2, 'val_number': 2}, {'writer': 'G. Golzi', 'val_agg': 0.3333333333333333, 'val_top3': 2, 'val_number': 6}, {'writer': 'A. Testa', 'val_agg': 0.18181818181818182, 'val_top3': 2, 'val_number': 11}, {'writer': 'Mogol', 'val_agg': 0.25, 'val_top3': 4, 'val_number': 16}, {'writer': 'P. Palma', 'val_agg': 1.0, 'val_top3': 2, 'val_number': 2}, {'writer': 'A. Salerno', 'val_agg': 0.5, 'val_top3': 4, 'val_number': 8}, {'writer': 'C. Mattone', 'val_agg': 0.2727272727272727, 'val_top3': 3, 'val_number': 11}, {'writer': 'G. Bigazzi', 'val_agg': 0.2222222222222222, 'val_top3': 4, 'val_number': 18}, {'writer': 'S. Bardotti', 'val_agg': 0.3333333333333333, 'val_top3': 2, 'val_number': 6}, {'writer': 'M. Fabrizio', 'val_agg': 0.2, 'val_top3': 3, 'val_number': 15}, {'writer': 'M. Masini', 'val_agg': 0.4, 'val_top3': 2, 'val_number': 5}, {'writer': 'A. Cogliati', 'va

In [40]:
aaa = sorted(tabella, key=lambda k: float(k['val_agg']), reverse=True)
bbb = sorted(tabella2, key=lambda k: float(k['val_agg']), reverse=True)
v_singers = ['D. Pace','F. Migliacci','M. Panzeri','Mogol','A. Cogliati']
print(aaa)
for s in v_singers:
    try:
        print(s,"  ", [a['writer']for a in aaa].index(s))
    except:
        print(s,"  ", "not found")

print("------------------")
for s in v_singers:
    try:
        print(s,"  ", aaa[[a['writer']for a in aaa].index(s)]["val_number"])
    except:
        print(s,"  ", bbb[[b['writer']for b in bbb].index(s)]["val_number"])

[{'writer': 'M. Luca', 'val_agg': 1.0, 'val_top3': 2, 'val_number': 2}, {'writer': 'P. Palma', 'val_agg': 1.0, 'val_top3': 2, 'val_number': 2}, {'writer': 'F. Ilacqua', 'val_agg': 1.0, 'val_top3': 2, 'val_number': 2}, {'writer': 'D. Verde', 'val_agg': 1.0, 'val_top3': 2, 'val_number': 2}, {'writer': 'F. Gabbani', 'val_agg': 1.0, 'val_top3': 2, 'val_number': 2}, {'writer': 'E. Ramazzotti', 'val_agg': 0.6, 'val_top3': 3, 'val_number': 5}, {'writer': 'A. Salerno', 'val_agg': 0.5, 'val_top3': 4, 'val_number': 8}, {'writer': 'D. Modugno', 'val_agg': 0.5, 'val_top3': 4, 'val_number': 8}, {'writer': 'G. Fiorelli', 'val_agg': 0.5, 'val_top3': 2, 'val_number': 4}, {'writer': 'M. Masini', 'val_agg': 0.4, 'val_top3': 2, 'val_number': 5}, {'writer': 'M. Falagiani', 'val_agg': 0.4, 'val_top3': 2, 'val_number': 5}, {'writer': 'L. Chiaravalli', 'val_agg': 0.4, 'val_top3': 2, 'val_number': 5}, {'writer': 'M. Ruccione', 'val_agg': 0.4, 'val_top3': 2, 'val_number': 5}, {'writer': 'G. Dati', 'val_agg': 0

Creation of the files

In [3]:
writers = open("artisti.csv","w")
writers.write("Id,Label\n")
writers.write('\n'.join([str(a) + "," + str(a) for a in lista]))#('\n'.join(lista))
writers.close()

writersTracks = open("artisti_e_canzoni.csv","w")
writersTracks.write("Writer,Songs\n")
writersTracks.write('\n'.join([str(a["writer"]) + "," + str(a["tracks"]) for a in dict_vett_writers]))
writersTracks.close()

outfile = open("edges_list.csv","w")
outfile.write("Source,Target,Weight\n")
outfile.write('\n'.join([str(list_edges[i]["source"]) + "," + str(list_edges[i]["target"]) +","+ str(list_weights[i]) for i in range( len(list_edges))]))
outfile.close()

Create a Graph

In [4]:
G = nx.Graph()
G.add_nodes_from(lista)
ebunch = [(list_edges[i]["source"],list_edges[i]["target"],{"weight":list_weights[i]}) for i in range( len(list_edges))]
G.add_edges_from(ebunch)

gMogol = nx.Graph()
gMogol.add_nodes_from(lista)
ebunchMogol = [(list_edges[i]["source"],list_edges[i]["target"],{"weight":list_weights[i]}) for i in range( len(list_edges)) if list_edges[i]["source"] == "Mogol" or list_edges[i]["target"] == "Mogol"]
gMogol.add_edges_from(ebunchMogol)
gMogol.remove_nodes_from(list(nx.isolates(gMogol)))

#degree centrality
print("degree centrality")
deg_centrality = nx.degree_centrality(G)
#nx.write_gexf(deg_centrality, "deg_cent.gexf")
print(deg_centrality,'\n')


#degree distribution
print("degree distribution")
print(nx.degree_histogram(G),'\n') #find probability

#pagerank
print("pagerank")
print(nx.pagerank(G, weight="weight"),'\n')

#average clustering coefficient
print("average clustering coefficient")
print(nx.average_clustering(G,weight="weight"),'\n')



#k-cliques
cliqueMatrix = []
cliqueMatrix.append([])
cliqueMatrix.append([])
for k in range(2,8):
    i = 0
    cliqueMatrix.append([])
    for clique in nx.find_cliques(G):
        if len(clique) == k:
            i += 1
            cliqueMatrix[k].append(clique)
        '''elif len(clique) > k:
            i += len(list(itertools.combinations(clique, k)))'''
    print(k,"-clique: ", i)
print('\n')

#clique edges experimental
def travel_list(l,k):
    for el in l:
        a = el.get(k,None)
        if a == None:
            continue
        else:
            return a

cliqueEdges= []
target = [{str(a["source"] + a["target"]):{"s":a["source"],"d":a["target"],"tracks":','.join([f["name"] + " - " + str(f["pos"]) for f in a["tracks"]])}} for a in l_e_new]
#print("target - ", target)
for i in range(len(cliqueMatrix[6])):
    cliqueEdges.append([])
    for node1 in cliqueMatrix[6][i]:
        s = []
        for node2 in cliqueMatrix[6][i]:
            t = travel_list(target,str(node1 + node2))
            if not t in s and t is not None:
                s.append(t)
        cliqueEdges.append(list(s))

result = [c for c in cliqueEdges ]#if len(c)>1]
#print("RES - ", result)

#############################
#find how many songs in every clique
cliqueSongs = []
for i in range(len(cliqueMatrix[6])):
    songs = []
    append_this = []
    for artist in cliqueMatrix[6][i]:
        for d in dict_vett_writers:
            if d["writer"] == artist:
                for song in d["tracks"]:
                    songs.append(song["name"] +" - " +str(song["pos"]))
        songs_set = set(songs)
        append_this = [[s,0] for s in songs_set]
        for song in songs:
            for j in range(len(append_this)):
                if song == append_this[j][0]:
                    append_this[j][1] += 1 #(song,append_this[j][1]+1)
        append_this = [a[0] for a in append_this if a[1] > 1]

    cliqueSongs.append("Clique number " + str(i) + " - " + str(append_this))
print("clique songs - ", cliqueSongs)

#############################

#closeness centrality
print("closeness centrality")
print(nx.closeness_centrality(G),'\n')

#betweenness centrality
print("betweenness centrality")
print(nx.betweenness_centrality(G,k=len(G.nodes()), normalized=True, weight="weight"),'\n')



#Degree Centrality, Degree Distibution, PageRank, Local Clustering Coefficient, N-cliques, Closeness, Betweenness

degree centrality
{'F. Monti Arduini': 0.0010162601626016261, 'M. Pecora': 0.0020325203252032522, 'S. Basile': 0.0, 'S. Belisari': 0.0040650406504065045, 'D. Miglietta': 0.003048780487804878, 'S. Sichel': 0.0020325203252032522, 'I. Ianne': 0.008130081300813009, 'E. Sciorilli': 0.005081300813008131, 'G. Chiocchio': 0.0010162601626016261, "V. D'Acquisto": 0.0010162601626016261, 'A. Di Zio': 0.0010162601626016261, 'L. Beccafichi': 0.0020325203252032522, 'F. Fasano': 0.0040650406504065045, 'G. Mangiaracina': 0.0020325203252032522, 'M. De Martino': 0.0010162601626016261, 'M. Marati': 0.011178861788617888, 'M. Marini': 0.0010162601626016261, 'W. Colì': 0.0, 'M. G. Gili': 0.003048780487804878, 'A. Civai': 0.003048780487804878, 'Lattuada': 0.0020325203252032522, 'V. Centrone': 0.0020325203252032522, 'Lubiak': 0.0010162601626016261, 'M. Mastrangeli': 0.0010162601626016261, 'E. Mineo': 0.0010162601626016261, 'L. Sala': 0.0020325203252032522, 'F. Maresca': 0.0010162601626016261, 'V. Venturi': 0.0

In [11]:
# sort by degree centrality
sorted_deg_centrality = sorted(deg_centrality.items(), key=operator.itemgetter(1), reverse=True)
print("sorted degree centrality")
print(sorted_deg_centrality[:5],'\n')

# sort by degree centrality
sorted_pagerank = sorted(nx.pagerank(G, weight="weight").items(), key=operator.itemgetter(1), reverse=True)
print("sorted pagerank")
print(sorted_pagerank[:5],'\n')

sorted degree centrality
[('D. Pace', 0.026422764227642278), ('F. Migliacci', 0.021341463414634148), ('M. Panzeri', 0.020325203252032523), ('Mogol', 0.01829268292682927), ('A. Cogliati', 0.016260162601626018)] 

sorted pagerank
[('D. Pace', 0.006650405993628742), ('F. Migliacci', 0.006273585517963275), ('Mogol', 0.005871133499830386), ('M. Panzeri', 0.004668332330600986), ('G. Bigazzi', 0.004577594346810482)] 



In [5]:
newfile = open("ce.txt","w")
newfile.write('\n'.join([str(b["s"]) + " | " + str(b["d"]) + " | " + str(b["tracks"]) for a in result for b in a])) 
newfile.close()

newfile2 = open("deg_centrality.csv","w")
newfile2.write("Writer,Label,Degree Centrality\n")
newfile2.write('\n'.join([str(a) + ","+ str(a) +"," + str(deg_centrality[a]) for a in deg_centrality]))
newfile2.close()

nx.write_gexf(gMogol, "mogol.gexf")
